In [1]:
import torch
!pip install -q torch-scatter~=2.1.0 torch-sparse~=0.6.16 torch-cluster~=1.6.0 torch-spline-conv~=1.2.1 torch-geometric==2.2.0 -f https://data.pyg.org/whl/torch-{torch.__version__}.html

torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.4/943.4 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [10]:
from torch import nn
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

In [13]:
class GCNConv(MessagePassing):
  def __init__(self, dim_in, hidden):
    super().__init__(aggr='add')
    self.linear = nn.Linear(dim_in, hidden, bias=False)

  def forward(self, x, edge_index):
    edge_index, _ = add_self_loops(edge_index, num_nudes=x.size(0))
    x = self.linear(x)
    row, col = edge_index
    deg = degree(col, x.size(0), dtype=x.dtype)
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
    norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

    out = self.propagete(edge_index, x=x, norm=norm)
    return out

  def message(self, x, norm):
    return norm.view(-1, 1) * x

In [17]:
conv = GCNConv(16, 32)

## Heterogenious Graphs

In [21]:
from torch_geometric.data import HeteroData

data = HeteroData()

data['user'].x = torch.Tensor([[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3]]) # feature vectors for user 1, 2, and 3: [num_users, num_features_users]
data['game'].x = torch.Tensor([[1, 1], [2, 2]]) # feature vectors for games
data['dev'].x = torch.Tensor([[1], [2]]) # feature vectors for devs

data['user', 'follows', 'user'].edge_index = torch.Tensor([[0, 1], [1, 2]]) # [2, num_edges_follows]
data['user', 'plays', 'game'].edge_index = torch.Tensor([[0, 1, 1, 2], [0, 0, 1, 1]])
data['dev', 'develops', 'game'].edge_index = torch.Tensor([[0, 1], [0, 1]])

data['user', 'plays', 'game'].edge_attr = torch.Tensor([[2], [0.5], [10], [12]])

data

HeteroData(
  user={ x=[3, 4] },
  game={ x=[2, 2] },
  dev={ x=[2, 1] },
  (user, follows, user)={ edge_index=[2, 2] },
  (user, plays, game)={
    edge_index=[2, 4],
    edge_attr=[4, 1]
  },
  (dev, develops, game)={ edge_index=[2, 2] }
)

## Classic GAT

In [22]:
from torch import nn
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import DBLP
from torch_geometric.nn import GAT

In [26]:
# define meta-path
metapaths = [[('author', 'paper'), ('paper', 'author')]]

# declare transform function
transform = T.AddMetaPaths(metapaths=metapaths, drop_orig_edge_types=True)

# load DBLP dataset
dataset = DBLP('.', transform=transform)
data = dataset[0]
print(data)

Extracting ./raw/DBLP_processed.zip
Processing...
Done!


HeteroData(
  metapath_dict={ (author, metapath_0, author)=[2] },
  author={
    x=[4057, 334],
    y=[4057],
    train_mask=[4057],
    val_mask=[4057],
    test_mask=[4057]
  },
  paper={ x=[14328, 4231] },
  term={ x=[7723, 50] },
  conference={ num_nodes=20 },
  (author, metapath_0, author)={ edge_index=[2, 11113] }
)


/usr/local/lib/python3.10/dist-packages/torch_sparse/matmul.py:97: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  C = torch.sparse.mm(A, B)


In [27]:
# model
model = GAT(in_channels=-1, hidden_channels=64, out_channels=4, num_layers=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)

@torch.inference_mode()
def test(mask):
  model.eval()
  pred = model(data.x_dict['author'], data.edge_index_dict[('author', 'metapath_0', 'author')]).argmax(dim=-1)
  acc = (pred[mask] == data['author'].y[mask]).sum() / mask.sum()
  return float(acc)

for epoch in range(101):
  model.train()
  optimizer.zero_grad()
  out = model(data.x_dict['author'], data.edge_index_dict[('author', 'metapath_0', 'author')])
  mask = data['author'].train_mask
  loss = F.cross_entropy(out[mask], data['author'].y[mask])
  loss.backward()
  optimizer.step()

  if epoch % 20 == 0:
    train_acc = test(data['author'].train_mask)
    val_acc = test(data['author'].val_mask)
    print(f'Epoch: {epoch:>3} | Train Loss:'
    f'{loss:.4f} | Train Acc: {train_acc*100:.2f}% | Val Acc: '
    f'{val_acc*100:.2f}%')

test_acc = test(data['author'].test_mask)
print(f'Test accuracy: {test_acc*100:.2f}%')

Epoch:   0 | Train Loss:1.3856 | Train Acc: 41.00% | Val Acc: 33.75%
Epoch:  20 | Train Loss:0.6273 | Train Acc: 87.75% | Val Acc: 75.50%
Epoch:  40 | Train Loss:0.4015 | Train Acc: 93.50% | Val Acc: 76.25%
Epoch:  60 | Train Loss:0.3031 | Train Acc: 95.75% | Val Acc: 75.75%
Epoch:  80 | Train Loss:0.2482 | Train Acc: 97.25% | Val Acc: 77.00%
Epoch: 100 | Train Loss:0.2129 | Train Acc: 98.50% | Val Acc: 75.75%
Test accuracy: 77.03%


## Heterogenious GAT

In [29]:
from torch_geometric.nn import GATConv, Linear, to_hetero
from torch import nn

dataset = DBLP(root='.')
data = dataset[0]

data['conference'].x = torch.zeros(20, 1)

In [36]:
# Heterogenious GAT

class GAT(nn.Module):
  def __init__(self, hidden, dim_out):
    super().__init__()
    self.conv = GATConv((-1, -1), hidden, add_self_loops=False)
    self.linear = nn.Linear(hidden, dim_out)

  def forward(self, x, edge_index):
    h = self.conv(x, edge_index).relu()
    h = self.linear(h)
    return h

In [38]:
model = GAT(hidden=64, dim_out=4)
model = to_hetero(model, data.metadata(), aggr='sum')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)

In [39]:
@torch.inference_mode()
def test(mask):
  model.eval()
  pred = model(data.x_dict, data.edge_index_dict)['author'].argmax(dim=-1)
  acc = (pred[mask] == data['author'].y[mask]).sum() / mask.sum()
  return float(acc)

In [40]:
for epoch in range(101):
  model.train()
  optimizer.zero_grad()
  out = model(data.x_dict, data.edge_index_dict)['author']
  mask = data['author'].train_mask
  loss = F.cross_entropy(out[mask], data['author'].y[mask])
  loss.backward()
  optimizer.step()

  if epoch % 20 == 0:
    train_acc = test(data['author'].train_mask)
    val_acc = test(data['author'].val_mask)
    print(f'Epoch: {epoch:>3} | Train Loss:'
    f'{loss:.4f} | Train Acc: {train_acc*100:.2f}% | Val Acc:'
    f'{val_acc*100:.2f}%')

test_acc = test(data['author'].test_mask)
print(f'Test accuracy: {test_acc*100:.2f}%')

Epoch:   0 | Train Loss:1.3870 | Train Acc: 37.50% | Val Acc:27.75%
Epoch:  20 | Train Loss:1.1953 | Train Acc: 92.50% | Val Acc:66.50%
Epoch:  40 | Train Loss:0.8572 | Train Acc: 97.00% | Val Acc:70.50%
Epoch:  60 | Train Loss:0.5095 | Train Acc: 99.00% | Val Acc:74.00%
Epoch:  80 | Train Loss:0.2721 | Train Acc: 99.50% | Val Acc:75.25%
Epoch: 100 | Train Loss:0.1540 | Train Acc: 100.00% | Val Acc:76.50%
Test accuracy: 78.17%


## HAN - Hierarchical Self-Attention Network

In [45]:
from torch_geometric.nn import HANConv

dataset = DBLP('.')
data = dataset[0]
data['conference'].x = torch.zeros(20, 1)

class HAN(nn.Module):
  def __init__(self, dim_in, dim_out, hidden=128, heads=8):
    super().__init__()
    self.han = HANConv(dim_in, hidden, heads=heads, dropout=0.6, metadata=data.metadata())
    self.linear = nn.Linear(hidden, dim_out)

  def forward(self, x_dict, edge_index_dict):
    out = self.han(x_dict, edge_index_dict)
    out = self.linear(out['author'])
    return out

model = HAN(dim_in=-1, dim_out=4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)

@torch.inference_mode()
def test(mask):
  model.eval()
  pred = model(data.x_dict, data.edge_index_dict).argmax(dim=-1)
  acc = (pred[mask] == data['author'].y[mask]).sum() / mask.sum()
  return float(acc)

In [46]:
for epoch in range(101):
  model.train()
  optimizer.zero_grad()
  out = model(data.x_dict, data.edge_index_dict)
  mask = data['author'].train_mask
  loss = F.cross_entropy(out[mask], data['author'].y[mask])
  loss.backward()
  optimizer.step()

  if epoch % 20 == 0:
    train_acc = test(data['author'].train_mask)
    val_acc = test(data['author'].val_mask)
    print(f'Epoch: {epoch:>3} | Train Loss:'
    f'{loss:.4f} | Train Acc: {train_acc*100:.2f}% | Val Acc:'
    f'{val_acc*100:.2f}%')

Epoch:   0 | Train Loss:1.3845 | Train Acc: 30.50% | Val Acc:26.25%
Epoch:  20 | Train Loss:1.1468 | Train Acc: 88.75% | Val Acc:66.25%
Epoch:  40 | Train Loss:0.7809 | Train Acc: 94.75% | Val Acc:70.00%
Epoch:  60 | Train Loss:0.4769 | Train Acc: 97.75% | Val Acc:75.25%
Epoch:  80 | Train Loss:0.3045 | Train Acc: 99.00% | Val Acc:78.50%
Epoch: 100 | Train Loss:0.2151 | Train Acc: 99.75% | Val Acc:79.00%


In [47]:
test_acc = test(data['author'].test_mask)
print(f'Test accuracy: {test_acc*100:.2f}%')

Test accuracy: 81.52%
